In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/liuda/Local/data/trading/data/community/reddit/community_reddit.csv')

In [3]:
df.columns

Index(['id', 'author_id', 'platform_id', 'publish_time', 'ticker_id', 'trend',
       'sentiment_score', 'reason', 'title', 'selftext', 'url', 'comment',
       'created_utc', 'clicked', 'num_comments', 'score', 'upvote_ratio',
       'link_flair_text', 'hotness', 'reddit_type'],
      dtype='object')

In [5]:
df['sentiment_score']

0      8
1      8
2      9
3      5
4      7
      ..
264    7
265    3
266    3
267    7
268    7
Name: sentiment_score, Length: 269, dtype: int64

In [7]:
ticker_df = df.groupby(['ticker_id', 'trend']).agg(
        avg_sentiment_score=('sentiment_score', lambda x: x.mean()),
).reset_index()


In [9]:
df = df.merge(ticker_df, on=['ticker_id', 'trend'], how='left', suffixes=['', '_y'])


In [11]:
drop_cols = [col for col in df.columns if col.endswith('_y')]
df = df.drop(columns=drop_cols)


In [12]:
df

,id,author_id,platform_id,publish_time,ticker_id,trend,sentiment_score,reason,title,selftext,...,comment,created_utc,clicked,num_comments,score,upvote_ratio,link_flair_text,hotness,reddit_type,avg_sentiment_score
0,1iz6io3,a14u4n60,reddit,2025-02-27 03:32:53,AAPL,Long,8,Commenters believe Apple will weather the stor...,Economic Black Out….. does this mean no buying...,Wonder how long it will take Apple to cave to ...,...,Dude you need to stop reading cnn lol\nwho car...,1.740627e+09,False,7,0,0.45,NaN,0.45,ticker,7.50
1,1iy48wi,vwpuvgyk,reddit,2025-02-25 20:00:29,AAPL,Long,8,The author believes AAPL looks good and predic...,"So many weak hands lol, know what you own, and...","[AAPL looks good, set for possible move to 260...",...,I agree,1.740514e+09,False,1,6,0.75,NaN,0.75,ticker,7.50
2,1ixzywv,1jwinv603s,reddit,2025-02-25 17:06:11,AAPL,Long,9,The article states that AAPL is holding strong...,UPV if you ADDED Shares of AAPL. AAPL HOLDING ...,NaN,...,NaN,1.740503e+09,False,0,0,0.25,NaN,0.25,ticker,7.50
3,1ixxqkb,fxia9how,reddit,2025-02-25 15:33:24,AAPL,Neutral,5,The comment mentions AAPL is up 36% for the ye...,The 5 most valuable public traded companies ar...,NaN,...,Remind me December 31st\n!remindme December 31...,1.740498e+09,False,17,21,0.86,NaN,0.86,ticker,4.25
4,1ixbs1p,1jwinv603s,reddit,2025-02-24 20:17:15,AAPL,Long,7,Many commenters predict AAPL will rise above $...,"AAPL at 247.90 today, do you think it's too pr...","[Potential test towards 260 - 270 ranges, pers...",...,"Buy and hold. \n\nIf it goes lower, buy even m...",1.740428e+09,False,17,22,1.00,NaN,1.00,ticker,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,1j04pap,6l4z3,reddit,2025-02-28 10:30:31,NVDA,Long,7,"The article believes NVDA is oversold, its val...",r/Stocks Daily Discussion & Fundamentals Frida...,"This is the daily discussion, so anything stoc...",...,Pretty shocking to see NVDA drop like a rock y...,1.740739e+09,False,6,3,1.00,NaN,1.00,stocks,7.50
265,1j07l80,el2wnqmv,reddit,2025-02-28 13:25:13,TSLA,Short,3,The commenters believe that the market is tank...,Fucked up market,Costco option just ripped me a new asshole any...,...,You still think orange man is here to help?\nD...,1.740749e+09,False,32,0,0.22,Advice,0.22,stocks,2.48
266,1j09s20,sjn12fgp7,reddit,2025-02-28 15:07:33,TSLA,Short,3,The article highlights a negative sentiment to...,TSLA shorts and puts are the most effective wa...,If you're an activist who hates Elon Musk and ...,...,Average TSLA volume is 81.97M. How many shares...,1.740755e+09,False,9,0,0.42,Advice,0.42,stocks,2.48
267,1j09c5p,qo8mp223u,reddit,2025-02-28 14:47:56,ZM,Long,7,The article predicts that Zoom will benefit fr...,Skype to Shut 14 Years After Microsoft’s $8.5 ...,>Microsoft Corp. is signaling the end of the l...,...,There is no use of owning two similar products...,1.740754e+09,False,40,134,0.96,NaN,0.96,stocks,7.00


In [ ]:
from flask import Flask, jsonify
import simplejson as json

app = Flask(__name__)

@app.route('/data')
def get_data():
    data = {
        'value1': 3.141592653589793,
        'value2': 2.718281828459045
    }

    # 使用 simplejson 的浮点数格式化
    response = app.response_class(
        response=json.dumps(data, indent=2, use_decimal=True, ignore_nan=True),
        status=200,
        mimetype='application/json'
    )
    return response


In [15]:
import json
from decimal import Decimal
from typing import Any

def round_floats(obj: Any, precision: int = 2):
    """递归地遍历数据结构，格式化所有浮点数并转换回 float"""
    if isinstance(obj, float):
        return float(round(Decimal(obj), precision))  # 确保转换回 float
    elif isinstance(obj, dict):
        return {k: round_floats(v, precision) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [round_floats(v, precision) for v in obj]
    return obj  # 其他类型不变

data = {
    "price": 3.14159,
    "details": {
        "value": 2.71828,
        "nested": {
            "rate": 1.23456
        }
    },
    "list": [9.8765, {"sub_value": 4.56789}]
}

formatted_data = round_floats(data, 2)
json_str = json.dumps(formatted_data, indent=4)
print(json_str)


{
    "price": 3.14,
    "details": {
        "value": 2.72,
        "nested": {
            "rate": 1.23
        }
    },
    "list": [
        9.88,
        {
            "sub_value": 4.57
        }
    ]
}


In [17]:
from decimal import Decimal

def format(value):
    obj = Decimal(value)
    x = float(round(obj, 2))  # 这里转换为 float 可能导致精度问题
    print(str(x))  # 可能出现多余的小数位

format("2.675")

2.68
